# Será que podemos comparar ideologias políticas a partir de tweets?

ideologia: conjunto de convicções filosóficas, sociais, políticas etc. de um indivíduo ou grupo de indivíduos.

Comparações:
    - análise gráfica comparativa da frequência de palavras ou relacionadas a tópicos: 
        religião, educação, orientação sexual, identidade de gênero, economia, saúde, porte de armas, combate a pobreza
            ex: Quantidade de tweets do Bolsonaro falando sobre drogas é muito maior que a do Lula, que mostra maior favor a política anti-drogas
            (Buscar se a quantidade de palavras segue a mesma tendência)

    - Comparação percentual entre palavras específicas em relação ao total

In [ ]:
# Imports section
import re
import pandas as pd
import unicodedata as uni

In [ ]:
def get_data():
    '''Extract the JSON data and save it in a dataframe'''
    # Open json files and read into dataframes
    with open('dataset/jairbolsonaro.json','r') as file_01:
        df_01=pd.read_json(file_01)
    with open('dataset/LulaOficial.json','r') as file_02:
        df_02=pd.read_json(file_02)
    # rename columns inplace
    df_01.rename(columns={'full_text':'Bolsonaro'}, inplace=True)
    df_02.rename(columns={'full_text':'Lula'}, inplace=True)
    # concatenate dataframes into one dataframe
    cols=[df_01['Bolsonaro'], df_02['Lula']]
    dataframe=pd.concat(cols,axis=1)

    return dataframe

In [ ]:
def clean(tweet):
    '''Normalizes every word, removes links, hashtags, numbers, mentions, pontuation, emogis and removes stopwords from a single tweet and returns a list with the remaining words'''
    # input as single tweet string
    # output as list of string words
    stopwords=('de', 'a', 'pra', 'fazer','estar','lula','none', 'o', 'que', 'd','e', 'do', 'da', 'em', 'um', 'nao', 'para', 'e', 'com', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'foi', 'ao', 'ele', 'das', 'tem', 'seu', 'sua', 'ou', 'ser', 'quando', 'muito', 'ha', 'nos', 'ja', 'esta', 'eu', 'também', 'so', 'pelo', 'pela', 'ate', 'isso', 'ela', 'entre', 'era', 'depois', 'sem', 'mesmo', 'aos', 'ter', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'estao', 'voce', 'tinha', 'foram', 'essa', 'num', 'nem', 'suas', 'meu', 'minha', 'numa', 'pelos', 'elas', 'havia', 'seja', 'qual', 'sera', 'tenho', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'fosse', 'dele', 'tu', 'te', 'voces', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'esta', 'estamos', 'estao', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estavamos', 'estavam', 'estivera', 'estiveramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivessemos', 'estivessem', 'estiver', 'estivermos', 'estiverem', 'hei', 'ha', 'havemos', 'hao', 'houve', 'houvemos', 'houveram', 'houvera', 'houveramos', 'haja', 'hajamos', 'hajam', 'houvesse', 'houvessemos', 'houvessem', 'houver', 'houvermos', 'houverem', 'houverei', 'houvera', 'houveremos', 'houverao', 'houveria', 'houveriamos', 'houveriam', 'sou', 'somos', 'sao', 'era', 'eramos', 'eram', 'fui', 'foi', 'fomos', 'foram', 'fora', 'foramos', 'seja', 'sejamos', 'sejam', 'fosse', 'fossemos', 'fossem', 'for', 'formos', 'forem', 'serei', 'sera', 'seremos', 'serao', 'seria', 'seriamos', 'seriam', 'tenho', 'tem', 'temos', 'tem', 'tinha', 'tinhamos', 'tinham', 'tive', 'teve', 'tivemos', 'tiveram', 'tivera', 'tiveramos', 'tenha', 'tenhamos', 'tenham', 'tivesse', 'tivessemos', 'tivessem', 'tiver', 'tivermos', 'tiverem', 'terei', 'tera', 'teremos', 'terao', 'teria', 'teriamos', 'teriam')
    if type(tweet) != str:
        tweet='none'
    # normalize text and make all letters lowercase
    tweet = uni.normalize('NFD', tweet).encode('ASCII', 'ignore').decode('utf-8').lower()
    
    # remove links
    url_pattern=r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'\".,<>?«»“”‘’]))'
    
    # remove hashtags
    tags_pattern=r'\#([a-zA-Z0-9_]{1,50})'
    
    # remove mentions
    mentions_pattern=r'\@([a-zA-Z0-9_]{1,50})'    
    
    # remove ponctuation, emogis and numbers and making sure there's only words in the output
    rest_pattern=r'[^\w\s]+|\d+'
    
    # apply all patterns deleting all matched objects
    # patterns = [url_pattern,tags_pattern,mentions_pattern,rest_pattern]
    # for pattern in patterns:
    #     tweet = re.sub(pattern,'',tweet)

    patterns=re.compile('|'.join([url_pattern,tags_pattern,mentions_pattern,rest_pattern]))

    tweet = re.sub(patterns,'',tweet)

    clean_tweet=[word for word in tweet.split() if word not in stopwords]
    
    return clean_tweet

In [ ]:
def count_words(tweets):
    '''Count the ocurrence of key words in the tweets'''
    words={}
    for tweet in tweets:
        for word in clean(tweet):
            if word in words.keys() and type(word)==str:
                words[word] += 1
            else:
                words[word] = 1

    return sorted(words.items(),key= lambda x:x[1], reverse=True)

In [ ]:
def filter_data(data,themes):
    '''filters the data based on selected themes'''
    result=[]
    for theme in themes:
        for set in data:
            if theme in set:
                result.append(set)
    return result


In [ ]:
data = get_data()

b_data = data['Bolsonaro'] #list of tweets
l_data = data['Lula'] #list of tweets

themes = ['saude', 'educacao', 'seguranca', 'economia', 'moradia', 'corrupcao', 'inclusao','armas','pobreza']

# both b_count and l_count have all words, filter_data() will select relevant themes from those lists
b_count = count_words(b_data) #list of sets of (keyword, ocurrence)
l_count = count_words(l_data) #list of sets of (keyword, ocurrence)

In [ ]:
d1 = filter_data(b_count,themes)
df1 = pd.DataFrame(d1,columns=['Palavra','Bolsonaro'])
d2 = filter_data(l_count,themes)
df2 = pd.DataFrame(d2,columns=['Palavra','Lula'])
df = df1.set_index('Palavra').join(df2.set_index('Palavra'))

In [ ]:
ax = df.plot.barh(color={'Bolsonaro':'blue','Lula':'red'})